In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, service=service)

driver.get('https://www.margxt.fr/le-pokedex-de-paldea-liste-des-pokemon-et-ou-les-trouver-dans-pokemon-ecarlate-et-violet/')

In [3]:
table = driver.find_element(By.TAG_NAME, 'table').find_element(By.TAG_NAME, 'tbody')

pokedex = []
for e in table.find_elements(By.TAG_NAME, 'tr'):
    poke = e.text.split('\n')[:2]
    poke[1] = poke[1].split(' ')[0]
    if poke[0].isnumeric():
        img = e.find_element(By.TAG_NAME, 'img').get_attribute('src')
        poke.append(img)
        pokedex.append(tuple(poke))

In [4]:
# Missing from table elements
pokedex[50] = ('51', 'Mustébouée', 'https://www.margxt.fr/wp-content/uploads/2022/02/Muste%CC%81boue%CC%81e.png')
pokedex[354] = ('355', 'Polarhume', 'https://www.margxt.fr/wp-content/uploads/2022/10/Polarhume.png')

In [5]:
df = pd.DataFrame(pokedex, columns=['num', 'name', 'image'])
df.to_csv('Pokédex de Paldea.csv', index=False)

In [12]:
_pokedex = [f'({x[0]})<br>{x[1]}<br><img decoding="async" loading="lazy" width="50" height="50" style="width: 50px;" src="{x[2]}" alt="">' for x in pokedex]

In [13]:
n = 30
pokedex_chunks = [_pokedex[i:i + n] for i in range(0, len(_pokedex), n)]

In [14]:
html_body = ''
html_body += '''<!doctype html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-GLhlTQ8iRABdZLl6O3oVMWSktQOp6b7In1Zl3/Jr59b6EGGoI1aFkw7cmDA6j6gD" crossorigin="anonymous">
  </head>
  <style type="text/css">
  table td:hover {
  background-color: rgba(2, 123, 255);
  }
  td {
  text-align: center;
  }
  th {
  text-align: center;
  }
  body {
  background-color: rgba(32, 37, 40, 0.96);
  }
  </style>

'''

In [15]:
for idx, chunk in enumerate(pokedex_chunks, start=1):
    n = 6
    l2 = [chunk[i:i + n] for i in range(0, len(chunk), n)]
    df = pd.DataFrame(l2)
    df.index += 1 
    df.columns += 1
    html_body += f'<h2 align="center"><span class="badge text-bg-danger">Boîte {idx}</span></h2>\n'
    html_body += df.to_html()

In [16]:
html_body += '''
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/js/bootstrap.bundle.min.js" integrity="sha384-w76AqPfDkMBDXo30jS1Sgez6pr3x5MlQ1ZAGC+nuZB+EYdgRZgiwxhTBTkF7CXvN" crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.11.6/dist/umd/popper.min.js" integrity="sha384-oBqDVmMz9ATKxIep9tiCxS/Z9fNfEXiDAYTujMAeBAsjFuCZSmKbSSUnQlmh/jp3" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/js/bootstrap.min.js" integrity="sha384-mQ93GR66B00ZXjt0YO5KlohRA5SY2XofN4zfuZxLkoj1gXtW8ANNCe9d5Y3eG5eD" crossorigin="anonymous"></script>
  </body>
  </html>
'''

html_body = html_body.replace('class="dataframe"', 'class="table table-dark"').replace('&lt;', '<').replace('&gt;', '>').replace('None', '')


In [17]:
with open('Pokédex de Paldea.html', 'w') as f:
    f.write(html_body)